In [69]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


### Данные

#### Загрузка

In [2]:
bio_data_orig = pd.read_csv('./_train_sem09__1_.zip')

bio_data_orig.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


#### Анализ

Согласно формулировке задания, никакая предварительная обработка данных не требуется.

Посмотрим лишь являются ли наши данные сбалансированными.

In [6]:
m1 = bio_data_orig['Activity'] == 1
print(f'Молекул с положительным биологическим ответом: {m1.sum()}')

m0 = bio_data_orig['Activity'] == 0
print(f'Молекул с отрицательным биологическим ответом: {m0.sum()}')

Молекул с положительным биологическим ответом: 2034
Молекул с отрицательным биологическим ответом: 1717


Данные можно считать сбалансированными.

#### Подготовка сетов

In [11]:
X = bio_data_orig.drop(columns=['Activity'])
y = bio_data_orig['Activity']

Не смотря на то, что данные являются сбалансированными, имеет смысл сделать стратифицированное разбиение.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(X_train.shape)
print(X_test.shape)

(3000, 1776)
(751, 1776)


### Обучение моделей

#### Логистическая регрессия

Создадим модель с параметрами по-умолчанию.

In [13]:
log_reg = linear_model.LogisticRegression(
    random_state=42,
)

log_reg.fit(X_train, y_train)

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Попробуем задать `max_iter` чтобы модель могла сойтись.

In [17]:
log_reg = linear_model.LogisticRegression(
    max_iter=500,
)

log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=500)

Оценим качество модели по метрике F1

In [20]:
y_ = y_test

y_pred = log_reg.predict(X_test)

print(f'F₁ score: {metrics.f1_score(y_, y_pred)}')

F₁ score: 0.7773766546329723


#### Случайный лес

Создадим модель с параметрами по-умолчанию.

In [25]:
rf_clf = ensemble.RandomForestClassifier(
    random_state=42
)

rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Оценим качество модели по метрике F1

In [26]:
y_ = y_test

y_pred = rf_clf.predict(X_test)

print(f'F₁ score: {metrics.f1_score(y_, y_pred)}')

F₁ score: 0.8048484848484848


### Предварительные результаты

Мы обучили две модели с параметрами по-умолчанию. Обе модели дали похожие результаты по метрике F1:

- Логистическая регрессия: 0.78
- Случайный лес: 0.80]

### Оптимизация гиперпараметров

#### GridSearchCV

##### Оптимизируем лог. регрессию

In [38]:
param_grid = [
    {
        'penalty': ['l2', None],
        'solver': ['lbfgs', 'newton', 'newton', 'sag'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
    },
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
    },
    {
        'penalty': ['elasticnet', 'l1', 'l2', None],
        'solver': ['saga'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
    },
]

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        # неизменяемые параметры
        random_state=42,
        max_iter=50,
        class_weight='balanced',
    ),
    scoring='f1',
    param_grid=param_grid,
    cv=5, # кол-во фолдов кросс-валдиации
    n_jobs = -1, # кол-во ядер
)

grid_search.fit(X_train, y_train);

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

In [37]:
print('Наилучшая комбинация гиперпараметров:')
print(grid_search.best_params_)

print('Метрика на тестовой выборке:')
print(grid_search.score(X_test, y_test))

Наилучшая комбинация гиперпараметров:
{'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}
Метрика на тестовой выборке:
0.7722289890377588


Нам не удалось улучшить метрику, но мы нашли комбинацию параметров, которая позволяет обучить модель за 50 циклов вместо 500 и дает почти такой же результат. Чтобы найти оптимальную комбинацию понадобилось 30 секунд.

##### Оптимизируем случайный лес

In [61]:
param_grid = [
    {
        'criterion': ['gini', 'entropy', 'log_loss'],
        'max_depth': [10, 20, 30],
        'max_features': [0.01, 0.03, 0.1],
        'min_samples_leaf': [1, 2, 3, 5],
        'n_estimators': [50, 100, 200],
    }
]

grid_search = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=42,
    ),
    scoring='f1',
    param_grid=param_grid,
    cv=5, # кол-во фолдов кросс-валдиации
    n_jobs = -1, # кол-во ядер
)

grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy', 'log_loss'],
                          'max_depth': [10, 20, 30],
                          'max_features': [0.01, 0.03, 0.1],
                          'min_samples_leaf': [1, 2, 3, 5],
                          'n_estimators': [50, 100, 200]}],
             scoring='f1')

In [63]:
print('Наилучшая комбинация гиперпараметров:')
print(grid_search.best_params_)

print('Метрика на тестовой выборке:')
print(grid_search.score(X_test, y_test))


Наилучшая комбинация гиперпараметров:
{'criterion': 'gini', 'max_depth': 30, 'max_features': 0.1, 'min_samples_leaf': 2, 'n_estimators': 100}
Метрика на тестовой выборке:
0.8057210965435043


Нам удалось немного улучшить метрику. На поиск лучшей комбинации гиперпараметров мы потратили почти 5 минут.

#### RandomizedSearchCV

##### Оптимизируем лог. регрессию

In [67]:
param_distributions = [
    {
        'penalty': ['l2', None],
        'solver': ['lbfgs', 'newton', 'newton', 'sag'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
    },
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
    },
    {
        'penalty': ['elasticnet', 'l1', 'l2', None],
        'solver': ['saga'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
    },
]

random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(
        # неизменяемые параметры
        random_state=42,
        max_iter=50,
        class_weight='balanced',
    ),
    scoring='f1',
    param_distributions=param_distributions,
    n_iter=10,
    cv=5, # кол-во фолдов кросс-валдиации
    n_jobs = -1, # кол-во ядер
)

random_search.fit(X_train, y_train);


/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.

In [68]:
print('Наилучшая комбинация гиперпараметров:')
print(random_search.best_params_)

print('Метрика на тестовой выборке:')
print(random_search.score(X_test, y_test))

Наилучшая комбинация гиперпараметров:
{'solver': 'saga', 'penalty': 'l1', 'C': 0.3}
Метрика на тестовой выборке:
0.7722289890377588


Нам не удалось улучшить метрику, но зато с помощью `RandomizedSearchCV` мы за 6 секунд нашли ту же комбинацию гиперпараметров, которую мы нашли за 30 секунд с помощью `GridSearchCV`.

##### Оптимизируем случайный лес

In [72]:
param_distributions = [
    {
        'criterion': ['gini', 'entropy', 'log_loss'],
        'max_depth': list(np.linspace(10, 50, 10, dtype=int)),
        'max_features': [0.01, 0.03, 0.1],
        'min_samples_leaf': list(np.linspace(1, 9, 5, dtype=int)),
        'n_estimators': list(np.linspace(50, 250, 6, dtype=int)),
    }
]

random_search = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=42,
    ),
    scoring='f1',
    param_distributions=param_distributions,
    n_iter=20,
    cv=5, # кол-во фолдов кросс-валдиации
    n_jobs = -1, # кол-во ядер
)

random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions=[{'criterion': ['gini', 'entropy',
                                                       'log_loss'],
                                         'max_depth': [10, 14, 18, 23, 27, 32,
                                                       36, 41, 45, 50],
                                         'max_features': [0.01, 0.03, 0.1],
                                         'min_samples_leaf': [1, 3, 5, 7, 9],
                                         'n_estimators': [50, 90, 130, 170, 210,
                                                          250]}],
                   scoring='f1')

In [73]:
print('Наилучшая комбинация гиперпараметров:')
print(random_search.best_params_)

print('Метрика на тестовой выборке:')
print(random_search.score(X_test, y_test))

Наилучшая комбинация гиперпараметров:
{'n_estimators': 170, 'min_samples_leaf': 3, 'max_features': 0.1, 'max_depth': 18, 'criterion': 'log_loss'}
Метрика на тестовой выборке:
0.8023668639053254


Метод проверил 20 коминаций, потратив на это 25 секунд, но не смог улучшить метрику.